In [1]:
import os
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from groq import Groq
import time
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Define workspace path
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"

# Set environment variables to store large files in workspace
os.environ["HF_HOME"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["TRANSFORMERS_CACHE"] = os.path.join(WORKSPACE_PATH, "huggingface")
os.environ["HF_DATASETS_CACHE"] = os.path.join(WORKSPACE_PATH, "datasets")
os.environ["NLTK_DATA"] = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure necessary directories exist
os.makedirs(os.environ["HF_HOME"], exist_ok=True)
os.makedirs(os.environ["TRANSFORMERS_CACHE"], exist_ok=True)
os.makedirs(os.environ["HF_DATASETS_CACHE"], exist_ok=True)
os.makedirs(os.environ["NLTK_DATA"], exist_ok=True)
BM25_INDEX_PATH = os.path.join(WORKSPACE_PATH, "bm25_index.pkl")
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
NLTK_PATH = os.path.join(WORKSPACE_PATH, "nltk_data")

# Ensure directory exists
os.makedirs(NLTK_PATH, exist_ok=True)

# Set NLTK data path
nltk.data.path.append(NLTK_PATH)

# Force download punkt & stopwords
nltk.download("punkt", download_dir=NLTK_PATH, force=True)
nltk.download("stopwords", download_dir=NLTK_PATH, force=True)
nltk.download("wordnet", download_dir=NLTK_PATH, force=True)
print(f"Workspace set up at: {WORKSPACE_PATH}")

[nltk_data] Downloading package punkt to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/horse/ws/uskh580e-myws/nltk_data...


Workspace set up at: /data/horse/ws/uskh580e-myws


In [3]:
DATASET_CACHE_DIR = os.path.join(WORKSPACE_PATH, "huggingface_datasets")

# Ensure the directory exists
os.makedirs(DATASET_CACHE_DIR, exist_ok=True)

# Load datasets and store them in workspace cache
query_data = load_dataset("princeton-nlp/LitSearch", "query", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full", cache_dir=DATASET_CACHE_DIR)
corpus_s2orc_data = load_dataset("princeton-nlp/LitSearch", "corpus_s2orc", split="full", cache_dir=DATASET_CACHE_DIR)

print("Datasets loaded successfully and stored in:", DATASET_CACHE_DIR)
query_df = query_data.to_pandas()
corpus_clean_df = corpus_clean_data.to_pandas()
corpus_s2orc_df = corpus_s2orc_data.to_pandas()
corpus_df = corpus_clean_df

Datasets loaded successfully and stored in: /data/horse/ws/uskh580e-myws/huggingface_datasets


In [4]:
class BM25Retriever:
    def __init__(self, corpus_df, text_column="abstract", stopword_removal=True, use_stemming=True):
        """
        BM25 Retriever for document retrieval.

        Parameters:
        - corpus_df: Pandas DataFrame containing the corpus
        - text_column: Column in corpus_df that contains the text for retrieval
        - stopword_removal: Whether to remove stopwords
        - use_stemming: Whether to apply stemming
        """
        self.corpus_df = corpus_df
        self.text_column = text_column
        self.stopword_removal = stopword_removal
        self.use_stemming = use_stemming
        self.bm25 = None
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words("english"))

    def tokenize_and_stem(self, text):
        """ Tokenize, remove stopwords, and apply stemming (if enabled). """
        tokens = word_tokenize(text.lower())  # Tokenize & lowercase
        if self.stopword_removal:
            tokens = [token for token in tokens if token not in self.stop_words]  # Remove stopwords
        if self.use_stemming:
            tokens = [self.stemmer.stem(token) for token in tokens]  # Apply stemming
        return tokens

    def create_index(self):
        """ Create BM25 index from the corpus. """
        print("Tokenizing and processing corpus...")
        self.corpus_df["tokens"] = self.corpus_df[self.text_column].astype(str).apply(self.tokenize_and_stem)
        self.bm25 = BM25Okapi(self.corpus_df["tokens"].tolist())

    def save_index(self, filename=BM25_INDEX_PATH):
        """ Save BM25 index to workspace. """
        with open(filename, "wb") as f:
            pickle.dump(self.bm25, f)
        print(f"BM25 index saved to {filename}")

    def load_index(self, filename=BM25_INDEX_PATH):
        """ Load BM25 index from workspace. """
        with open(filename, "rb") as f:
            self.bm25 = pickle.load(f)
        print(f"BM25 index loaded from {filename}")

    def retrieve(self, query, k=10):
        """ Retrieve top-k documents for a query. """
        tokenized_query = self.tokenize_and_stem(query)
        scores = self.bm25.get_scores(tokenized_query)
        ranked_indices = np.argsort(scores)[::-1][:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

In [5]:
from groq import Groq
import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm


# Initialize BM25 Retriever
bm25_retriever = BM25Retriever(corpus_df)

# ✅ Ensure BM25 index is loaded or created
if os.path.exists(BM25_INDEX_PATH):
    print("✅ Loading existing BM25 index...")
    bm25_retriever.load_index(BM25_INDEX_PATH)

# If BM25 index is still None, create it
if bm25_retriever.bm25 is None:
    print("⚠️ BM25 index not found, creating a new index...")
    bm25_retriever.create_index()
    bm25_retriever.save_index(BM25_INDEX_PATH)


# Initialize Groq API client
groq_client = Groq(api_key="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")

def reformulate_query(query, strategy):
    """ Reformulates the query only for LLM input, not for retrieval. """
    if strategy == "zero-shot":
        return query  # No modification, use as-is.

    elif strategy == "few-shot":
        return f"""Here are examples of how to answer questions while citing sources from retrieved documents.

        Example 1:
        Question: How do transformer models compare to LSTMs?
        Answer: Transformer models outperform LSTMs due to their ability to capture long-range dependencies. 
        (Source: "Transformer Models for NLP", Corpus ID: 12345)

        Example 2:
        Question: What is the role of reinforcement learning in AI?
        Answer: Reinforcement learning is widely used in robotics, game playing, and automated control tasks. 
        (Source: "Reinforcement Learning in AI Applications", Corpus ID: 67890)

        Example 3:
        Question: How effective are BERT embeddings for document retrieval?
        Answer: BERT embeddings improve retrieval performance by encoding semantic similarity, outperforming traditional TF-IDF methods.
        (Source: "Advancements in Document Retrieval using BERT", Corpus ID: 11223)

        Now, using the retrieved documents below, answer the question while citing sources explicitly:
        {query}"""

    elif strategy == "chain-of-thought":
        return f"""Let's break down the question step by step:

        Step 1: Identify key concepts and entities in the question.
        Step 2: Search for relevant supporting evidence from retrieved documents.
        Step 3: Construct an answer using the evidence while explicitly citing the relevant document IDs.

        Now, process this query:
        {query}"""


def generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_texts):
    """ Generates an answer using RAG, keeping retrieval separate from query modification. """
    context = "\n".join([
        f"Document ID: {doc_id}\nTitle: {title}\nFull Text: {full_text}"
        for doc_id, title, full_text in zip(retrieved_corpus_ids, retrieved_titles, retrieved_texts)
    ])

    prompt = f"""Use the following retrieved documents to answer the question.

    Retrieved Documents:
    {context}

    Question: {modified_query}
    Answer:"""

    response = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are an AI that generates well-supported answers using retrieved documents."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content  # Return the generated answer


# Ensure `query_df` has ground-truth corpus IDs stored as lists
query_df["corpusids"] = query_df["corpusids"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Use the entire dataset for retrieval
query_sample_df = query_df

# ✅ **Run RAG without altering the retrieval query**
rag_results = []

for _, row in tqdm(query_sample_df.iterrows(), total=len(query_sample_df), desc="Running RAG with different retrievals per strategy"):
    original_query = row["query"]
    ground_truth_corpus_ids = row["corpusids"]  # Retrieve the ground-truth relevant corpus IDs

    # **Retrieve using the original query (NO query modification before retrieval)**
    retrieved_ids = bm25_retriever.retrieve(original_query, k=10)
    
    # Fetch retrieved document details
    ranked_retrieved_df = pd.DataFrame({"corpusid": retrieved_ids})
    retrieved_docs_info = ranked_retrieved_df.merge(
        corpus_df[["corpusid", "title", "abstract"]], 
        on="corpusid", 
        how="left"
    )
    retrieved_corpus_ids = retrieved_docs_info["corpusid"].tolist()
    retrieved_titles = retrieved_docs_info["title"].tolist()
    retrieved_abstracts = retrieved_docs_info["abstract"].tolist()

    # ✅ **Now apply different prompting strategies (AFTER retrieval)**
    for strategy in ["zero-shot", "few-shot", "chain-of-thought"]:
        modified_query = reformulate_query(original_query, strategy)

        # Generate answer using the modified query (but retrieved docs stay the same)
        generated_answer = generate_with_rag(original_query, modified_query, retrieved_corpus_ids, retrieved_titles, retrieved_abstracts)

        # Identify which corpus IDs are actually cited in the answer
        cited_corpus_ids = [str(doc_id) for doc_id in retrieved_corpus_ids if str(doc_id) in generated_answer]

        # Store results
        rag_results.append({
            "query": original_query,
            "strategy": strategy,
            "modified_query": modified_query,
            "retrieved_corpus_ids": retrieved_corpus_ids,  # Documents retrieved by BM25
            "retrieved": retrieved_abstracts,  # Full text of retrieved documents
            "generated_answer": generated_answer,
            "cited_corpus_ids": cited_corpus_ids,  # Docs explicitly mentioned in the answer
            "ground_truth_corpus_ids": ground_truth_corpus_ids  # Actual relevant corpus IDs
        })

# Convert to DataFrame
rag_results_df = pd.DataFrame(rag_results)

# Save results to workspace
RAG_RESULTS_PATH = os.path.join(WORKSPACE_PATH, "rag_results_no_query_change.csv")
rag_results_df.to_csv(RAG_RESULTS_PATH, index=False)
display(rag_results_df)


✅ Loading existing BM25 index...
BM25 index loaded from /data/horse/ws/uskh580e-myws/bm25_index.pkl


Running RAG with different retrievals per strategy: 100%|█████████████████████████████| 597/597 [49:25<00:00,  4.97s/it]


,query,strategy,modified_query,retrieved_corpus_ids,retrieved,generated_answer,cited_corpus_ids,ground_truth_corpus_ids
0,Are there any research papers on methods to co...,zero-shot,Are there any research papers on methods to co...,"[3643430, 257038997, 256461337, 235294276, 248...",[Deep learning networks have achieved state-of...,"Yes, there are several research papers that fo...","[257038997, 248227350, 221995575, 234482764, 2...",[202719327]
1,Are there any research papers on methods to co...,few-shot,Here are examples of how to answer questions w...,"[3643430, 257038997, 256461337, 235294276, 248...",[Deep learning networks have achieved state-of...,"Yes, there are research papers on methods to c...","[257038997, 234482764, 219176798]",[202719327]
2,Are there any research papers on methods to co...,chain-of-thought,Let's break down the question step by step:\n\...,"[3643430, 257038997, 256461337, 235294276, 248...",[Deep learning networks have achieved state-of...,"To answer the question, let's break it down st...","[257038997, 248227350, 221995575, 234482764]",[202719327]
3,Are there any resources available for translat...,zero-shot,Are there any resources available for translat...,"[199379793, 227231792, 16825250, 15225095, 252...",[Parallel corpora available for building machi...,"Based on the retrieved documents, there are se...","[227231792, 16825250, 233365022]",[227231792]
4,Are there any resources available for translat...,few-shot,Here are examples of how to answer questions w...,"[199379793, 227231792, 16825250, 15225095, 252...",[Parallel corpora available for building machi...,"Yes, there are resources available for transla...","[227231792, 233365022, 16563009]",[227231792]
...,...,...,...,...,...,...,...,...
1786,What paper provides generalization bounds for ...,few-shot,Here are examples of how to answer questions w...,"[238582773, 257482844, 254247299, 248118561, 2...","[Recently, large-scale Contrastive Language-Im...","According to the retrieved documents, specific...","[238582773, 254247299, 247447209]",[263830433]
1787,What paper provides generalization bounds for ...,chain-of-thought,Let's break down the question step by step:\n\...,"[238582773, 257482844, 254247299, 248118561, 2...","[Recently, large-scale Contrastive Language-Im...","Based on the retrieved documents, I found no s...","[257482844, 254247299, 247447209]",[263830433]
1788,Which paper systematically examed the input mi...,zero-shot,Which paper systematically examed the input mi...,"[256390009, 261276856, 261682404, 247763065, 2...",[We explore the problem of generating minority...,The paper that systematically examined the inp...,[261276856],[261276856]
1789,Which paper systematically examed the input mi...,few-shot,Here are examples of how to answer questions w...,"[256390009, 261276856, 261682404, 247763065, 2...",[We explore the problem of generating minority...,Answer: The paper that systematically examined...,[261276856],[261276856]


In [6]:
def compute_metrics(df):
    metrics = []
    
    for strategy in df["strategy"].unique():
        subset = df[df["strategy"] == strategy]

        recall_5 = subset.apply(lambda row: len(set(row["retrieved_corpus_ids"][:5]) & set(row["ground_truth_corpus_ids"])) / len(row["ground_truth_corpus_ids"]) if len(row["ground_truth_corpus_ids"]) > 0 else 0, axis=1).mean()
        recall_20 = subset.apply(lambda row: len(set(row["retrieved_corpus_ids"][:20]) & set(row["ground_truth_corpus_ids"])) / len(row["ground_truth_corpus_ids"]) if len(row["ground_truth_corpus_ids"]) > 0 else 0, axis=1).mean()
        citation_precision = subset.apply(lambda row: len(set(row["cited_corpus_ids"]) & set(row["ground_truth_corpus_ids"])) / len(row["cited_corpus_ids"]) if len(row["cited_corpus_ids"]) > 0 else 0, axis=1).mean()
        citation_recall = subset.apply(lambda row: len(set(row["cited_corpus_ids"]) & set(row["ground_truth_corpus_ids"])) / len(row["ground_truth_corpus_ids"]) if len(row["ground_truth_corpus_ids"]) > 0 else 0, axis=1).mean()

        metrics.append({
            "Strategy": strategy,
            "Recall@5": recall_5,
            "Recall@20": recall_20,
            "Citation Precision": citation_precision,
            "Citation Recall": citation_recall
        })
    
    return pd.DataFrame(metrics)

# Compute and display the metrics
metrics_df = compute_metrics(rag_results_df)
display(metrics_df)

,Strategy,Recall@5,Recall@20,Citation Precision,Citation Recall
0,zero-shot,0.25215,0.457398,0.0,0.0
1,few-shot,0.25215,0.457398,0.0,0.0
2,chain-of-thought,0.25215,0.457398,0.0,0.0


In [11]:
import pandas as pd
import ast

# ✅ Ensure lists are properly formatted and contain only strings
for col in ["retrieved_corpus_ids", "ground_truth_corpus_ids", "cited_corpus_ids"]:
    rag_results_df[col] = rag_results_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    rag_results_df[col] = rag_results_df[col].apply(lambda x: list(map(str, x)) if isinstance(x, list) else [])

# ✅ Compute all retrieval and citation metrics
def compute_metrics(row):
    retrieved_corpus_ids = set(row["retrieved_corpus_ids"][:20])  # Ensure only top-20 retrieved
    ground_truth_corpus_ids = set(row["ground_truth_corpus_ids"])
    cited_corpus_ids = set(row["cited_corpus_ids"])

    tp_retrieved_5 = len(set(row["retrieved_corpus_ids"][:5]) & ground_truth_corpus_ids)
    tp_retrieved_20 = len(retrieved_corpus_ids & ground_truth_corpus_ids)
    tp_cited = len(cited_corpus_ids & ground_truth_corpus_ids)

    recall_5 = tp_retrieved_5 / len(ground_truth_corpus_ids) if len(ground_truth_corpus_ids) > 0 else 0.0
    recall_20 = tp_retrieved_20 / len(ground_truth_corpus_ids) if len(ground_truth_corpus_ids) > 0 else 0.0
    citation_precision = tp_cited / len(cited_corpus_ids) if len(cited_corpus_ids) > 0 else 0.0
    citation_recall = tp_cited / len(ground_truth_corpus_ids) if len(ground_truth_corpus_ids) > 0 else 0.0

    return pd.Series([recall_5, recall_20, citation_precision, citation_recall], 
                     index=["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"])

# ✅ Apply function
rag_results_df[["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"]] = rag_results_df.apply(compute_metrics, axis=1)

# ✅ Aggregate metrics per strategy
metrics_summary = rag_results_df.groupby("strategy")[
    ["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"]
].mean().reset_index()

# ✅ Display final table
display(metrics_summary)




,strategy,Recall@5,Recall@20,Citation Precision,Citation Recall
0,chain-of-thought,0.25215,0.457398,0.218511,0.371413
1,few-shot,0.25215,0.457398,0.251035,0.357677
2,zero-shot,0.25215,0.457398,0.237935,0.345449


In [13]:
from sentence_transformers import SentenceTransformer

/data/horse/ws/uskh580e-llm_citation_ws/myenv/lib/python3.10/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [15]:
import torch

In [21]:
from gritlm import GritLM

In [17]:
# Define workspace path
WORKSPACE_PATH = "/data/horse/ws/uskh580e-myws"
os.makedirs(WORKSPACE_PATH, exist_ok=True)

# ✅ Define paths for saving/retrieving cached embeddings
E5_INDEX_PATH = os.path.join(WORKSPACE_PATH, "e5_index.pkl")
GTR_INDEX_PATH = os.path.join(WORKSPACE_PATH, "gtr_index.pkl")
GRIT_INDEX_PATH = os.path.join(WORKSPACE_PATH, "grit_index.pkl")


In [18]:
class E5Retriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = SentenceTransformer("intfloat/e5-large-v2", device="cuda" if torch.cuda.is_available() else "cpu")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save E5 embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing E5 embeddings...")
        texts = ["passage: " + text for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=256, normalize_embeddings=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(E5_INDEX_PATH):
            try:
                with open(E5_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached E5 embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(E5_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        query_embedding = self.model.encode(["query: " + query], normalize_embeddings=True)
        scores = np.dot(query_embedding, self.embeddings.T)
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
e5_retriever = E5Retriever(corpus_df)

if e5_retriever.embeddings is None:
    print("⚠️ E5 embeddings not found, computing...")
    e5_retriever._prepare_embeddings()


✅ Loaded cached E5 embeddings


In [19]:
class GTRRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = SentenceTransformer("sentence-transformers/gtr-t5-large", device="cuda" if torch.cuda.is_available() else "cpu")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GTR embeddings if not cached."""
        if self._try_load_cache():
            return
        
        print("🔄 Computing GTR embeddings...")
        texts = self.corpus_df["abstract"].astype(str)
        self.embeddings = self.model.encode(texts, batch_size=256, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GTR_INDEX_PATH):
            try:
                with open(GTR_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GTR embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GTR_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        query_embedding = self.model.encode([query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
gtr_retriever = GTRRetriever(corpus_df)

if gtr_retriever.embeddings is None:
    print("⚠️ GTR embeddings not found, computing...")
    gtr_retriever._prepare_embeddings()


✅ Loaded cached GTR embeddings


In [22]:
class GRITRetriever:
    def __init__(self, corpus_df):
        self.corpus_df = corpus_df
        self.model = GritLM("GritLM/GritLM-7B", device_map="auto", torch_dtype=torch.bfloat16, mode="embedding")
        self.embeddings = None
        self._prepare_embeddings()

    def _prepare_embeddings(self):
        """Compute and save GRITLM embeddings if not cached."""
        if self._try_load_cache():
            return

        print("🔄 Computing GRIT embeddings...")
        texts = [f"<|embed|>\n{text}" for text in self.corpus_df["abstract"].astype(str)]
        self.embeddings = self.model.encode(texts, batch_size=128, convert_to_numpy=True, show_progress_bar=True)
        self._save_cache()

    def _try_load_cache(self):
        """Load cached embeddings if available."""
        if os.path.exists(GRIT_INDEX_PATH):
            try:
                with open(GRIT_INDEX_PATH, "rb") as f:
                    self.embeddings = pickle.load(f)
                    print("✅ Loaded cached GRIT embeddings")
                    return True
            except:
                print("⚠️ Failed to load cache.")
        return False

    def _save_cache(self):
        """Save embeddings to cache."""
        with open(GRIT_INDEX_PATH, "wb") as f:
            pickle.dump(self.embeddings, f)

    def retrieve(self, query, k=10):
        """Retrieve top-k documents for a query."""
        formatted_query = f"<|user|>\nRepresent this query for retrieving relevant documents:\n<|embed|>\n{query}"
        query_embedding = self.model.encode([formatted_query], convert_to_numpy=True)
        scores = query_embedding @ self.embeddings.T
        ranked_indices = np.argsort(-scores[0])[:k]
        return self.corpus_df.iloc[ranked_indices]["corpusid"].tolist()

# ✅ **Ensure Retrieval Occurs Before Query Reformulation**
grit_retriever = GRITRetriever(corpus_df)

if grit_retriever.embeddings is None:
    print("⚠️ GRITLM embeddings not found, computing...")
    grit_retriever._prepare_embeddings()


A new version of the following files was downloaded from https://huggingface.co/GritLM/GritLM-7B:
- modeling_gritlm7b.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Created GritLM: torch.bfloat16 dtype, mean pool, embedding mode, bbcc attn
✅ Loaded cached GRIT embeddings


In [54]:
import os
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
from groq import Groq

# ✅ **Initialize Retrievers**
retrievers = {
    "E5": e5_retriever,
    "GTR": gtr_retriever,
    "GRITLM": grit_retriever
}

# ✅ **Ensure Ground-Truth IDs are Correctly Formatted**
query_df["corpusids"] = query_df["corpusids"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# ✅ **Sample Only 10% of the Dataset**
query_sample_df = query_df.sample(frac=0.10, random_state=42)  # Randomly select 10% of rows

# ✅ **Run Retrieval, Then Reformulate Query and Run RAG**
rag_results = []

for retriever_name, retriever in retrievers.items():
    print(f"\n🚀 Running Retrieval with {retriever_name} on 10% dataset...\n")

    for _, row in tqdm(query_sample_df.iterrows(), total=len(query_sample_df), desc=f"Retrieving with {retriever_name}"):
        original_query = row["query"]
        ground_truth_corpus_ids = row["corpusids"]

        # **Step 1: Retrieve First, Before Query Reformulation**
        retrieved_ids = retriever.retrieve(original_query, k=20)
        retrieved_ids_for_llm = retrieved_ids[:10]
        
        # **Fetch retrieved document details**
        ranked_retrieved_df = pd.DataFrame({"corpusid": retrieved_ids})
        retrieved_docs_info = ranked_retrieved_df.merge(
            corpus_df[["corpusid", "title", "abstract"]], 
            on="corpusid", 
            how="left"
        )
        retrieved_corpus_ids = retrieved_docs_info["corpusid"].tolist()
        retrieved_titles = retrieved_docs_info["title"].tolist()
        retrieved_abstracts = retrieved_docs_info["abstract"].tolist()

        # **Step 2: Reformulate Query AFTER Retrieval**
        for strategy in ["zero-shot", "few-shot", "chain-of-thought"]:
            modified_query = reformulate_query(original_query, strategy)

            # **Step 3: Run LLM with Retrieved Documents**
            generated_answer = generate_with_rag(original_query, modified_query, retrieved_ids_for_llm, retrieved_titles[:10], retrieved_abstracts[:10])

            # **Step 4: Extract Cited Corpus IDs from LLM Response**
            cited_corpus_ids = [str(doc_id) for doc_id in retrieved_ids_for_llm if str(doc_id) in generated_answer]

            # **Step 5: Store Results**
            rag_results.append({
                "Retriever": retriever_name,
                "Query": original_query,
                "Strategy": strategy,
                "Retrieved Corpus IDs": retrieved_corpus_ids,
                "Retrieved Abstracts": retrieved_abstracts,
                "Generated Answer": generated_answer,
                "Cited Corpus IDs": cited_corpus_ids,
                "Ground Truth Corpus IDs": ground_truth_corpus_ids
            })

# ✅ Convert Results to DataFrame
rag_results_df = pd.DataFrame(rag_results)

# ✅ Save Results
RAG_RESULTS_PATH = os.path.join(WORKSPACE_PATH, "rag_results_e5_gtr_gritlm_10percent.csv")
rag_results_df.to_csv(RAG_RESULTS_PATH, index=False)
display(rag_results_df)

print(f"✅ Results saved at: {RAG_RESULTS_PATH}")




🚀 Running Retrieval with E5 on 10% dataset...



Retrieving with E5: 100%|███████████████████████████████████████████████████████████████| 60/60 [05:08<00:00,  5.14s/it]



🚀 Running Retrieval with GTR on 10% dataset...



Retrieving with GTR: 100%|██████████████████████████████████████████████████████████████| 60/60 [05:14<00:00,  5.24s/it]



🚀 Running Retrieval with GRITLM on 10% dataset...



Retrieving with GRITLM: 100%|███████████████████████████████████████████████████████████| 60/60 [05:28<00:00,  5.47s/it]


,Retriever,Query,Strategy,Retrieved Corpus IDs,Retrieved Abstracts,Generated Answer,Cited Corpus IDs,Ground Truth Corpus IDs
0,E5,Can you recommend research that uses an LLM to...,zero-shot,"[258041372, 252716013, 260125086, 253098700, 2...",[Current literature demonstrates that Large La...,"Based on the retrieved documents, here are som...","[252716013, 253098700, 264426355, 252596001, 2...",[229923710]
1,E5,Can you recommend research that uses an LLM to...,few-shot,"[258041372, 252716013, 260125086, 253098700, 2...",[Current literature demonstrates that Large La...,"According to the retrieved documents, a resear...","[261696510, 248986239, 258558102]",[229923710]
2,E5,Can you recommend research that uses an LLM to...,chain-of-thought,"[258041372, 252716013, 260125086, 253098700, 2...",[Current literature demonstrates that Large La...,Here's the breakdown of the question step by s...,"[258041372, 252716013, 260125086, 253098700, 2...",[229923710]
3,E5,Which paper proposed decomposing the logit upd...,zero-shot,"[254926669, 253761014, 247315171, 182953113, 1...",[Transformer-based large language models are t...,"Based on the retrieved documents, I couldn't f...","[254926669, 247315171, 182953113, 199552244, 4...",[258832652]
4,E5,Which paper proposed decomposing the logit upd...,few-shot,"[254926669, 253761014, 247315171, 182953113, 1...",[Transformer-based large language models are t...,"According to the retrieved documents, the pape...",[247315171],[258832652]
...,...,...,...,...,...,...,...,...
535,GRITLM,Could you recommend research that assesses tec...,few-shot,"[235097394, 250390904, 247596835, 239015827, 2...",[The ease of access to pre-trained transformer...,"Answer: Yes, there are several research papers...","[239015827, 260063038, 251928966, 259095603]",[237593027]
536,GRITLM,Could you recommend research that assesses tec...,chain-of-thought,"[235097394, 250390904, 247596835, 239015827, 2...",[The ease of access to pre-trained transformer...,"To answer your question, let's break down the ...","[250390904, 251928966, 256105128]",[237593027]
537,GRITLM,Which dataset supports narration generation an...,zero-shot,"[253628210, 252624427, 258832605, 8994087, 223...",[This paper constructs a Chinese dialoguebased...,"According to the retrieved documents, none of ...","[253628210, 258832605]",[258832605]
538,GRITLM,Which dataset supports narration generation an...,few-shot,"[253628210, 252624427, 258832605, 8994087, 223...",[This paper constructs a Chinese dialoguebased...,Answer: The dataset CMDQA supports narration g...,[253628210],[258832605]


✅ Results saved at: /data/horse/ws/uskh580e-myws/rag_results_e5_gtr_gritlm_10percent.csv


In [56]:
RAG_RESULTS_PATH = os.path.join(WORKSPACE_PATH, "rag_results_e5_gtr_gritlm_10percent.csv")

# Load the CSV file into a DataFrame
rag_results_df = pd.read_csv(RAG_RESULTS_PATH)

In [57]:
rag_results_df.head()

,Retriever,Query,Strategy,Retrieved Corpus IDs,Retrieved Abstracts,Generated Answer,Cited Corpus IDs,Ground Truth Corpus IDs
0,E5,Can you recommend research that uses an LLM to...,zero-shot,"[258041372, 252716013, 260125086, 253098700, 2...",['Current literature demonstrates that Large L...,"Based on the retrieved documents, here are som...","['252716013', '253098700', '264426355', '25259...",[229923710]
1,E5,Can you recommend research that uses an LLM to...,few-shot,"[258041372, 252716013, 260125086, 253098700, 2...",['Current literature demonstrates that Large L...,"According to the retrieved documents, a resear...","['261696510', '248986239', '258558102']",[229923710]
2,E5,Can you recommend research that uses an LLM to...,chain-of-thought,"[258041372, 252716013, 260125086, 253098700, 2...",['Current literature demonstrates that Large L...,Here's the breakdown of the question step by s...,"['258041372', '252716013', '260125086', '25309...",[229923710]
3,E5,Which paper proposed decomposing the logit upd...,zero-shot,"[254926669, 253761014, 247315171, 182953113, 1...",['Transformer-based large language models are ...,"Based on the retrieved documents, I couldn't f...","['254926669', '247315171', '182953113', '19955...",[258832652]
4,E5,Which paper proposed decomposing the logit upd...,few-shot,"[254926669, 253761014, 247315171, 182953113, 1...",['Transformer-based large language models are ...,"According to the retrieved documents, the pape...",['247315171'],[258832652]


In [58]:
# ✅ Compute retrieval and citation metrics with debugging
def compute_metrics(row):
    retrieved_corpus_ids = set(row["Retrieved Corpus IDs"][:20])  # Top-20 retrieved documents
    ground_truth_corpus_ids = set(row["Ground Truth Corpus IDs"])
    cited_corpus_ids = set(row["Cited Corpus IDs"])

    # ✅ Debugging: Print if there's a mismatch
    if not retrieved_corpus_ids & ground_truth_corpus_ids:
        print(f"⚠️ No Overlap Detected for Query: {row['Query']}")
        print(f"Retrieved: {retrieved_corpus_ids}")
        print(f"Ground Truth: {ground_truth_corpus_ids}\n")

    if not ground_truth_corpus_ids:
        return pd.Series([0.0, 0.0, 0.0, 0.0], index=["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"])

    tp_retrieved_5 = len(set(row["Retrieved Corpus IDs"][:5]) & ground_truth_corpus_ids)
    tp_retrieved_20 = len(retrieved_corpus_ids & ground_truth_corpus_ids)
    tp_cited = len(cited_corpus_ids & ground_truth_corpus_ids)

    recall_5 = tp_retrieved_5 / len(ground_truth_corpus_ids) if ground_truth_corpus_ids else 0.0
    recall_20 = tp_retrieved_20 / len(ground_truth_corpus_ids) if ground_truth_corpus_ids else 0.0
    citation_precision = tp_cited / len(cited_corpus_ids) if cited_corpus_ids else 0.0
    citation_recall = tp_cited / len(ground_truth_corpus_ids) if ground_truth_corpus_ids else 0.0

    return pd.Series([recall_5, recall_20, citation_precision, citation_recall], 
                     index=["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"])

# ✅ Apply function
rag_results_df[["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"]] = rag_results_df.apply(compute_metrics, axis=1)

# ✅ Aggregate metrics per strategy and retriever
metrics_summary = rag_results_df.groupby(["Retriever", "Strategy"])[
    ["Recall@5", "Recall@20", "Citation Precision", "Citation Recall"]
].mean().reset_index()

# ✅ Display final table
display(metrics_summary)

# ✅ Save to CSV
METRICS_RESULTS_PATH = os.path.join(WORKSPACE_PATH, "retrieval_metrics_e5_gtr_gritlm_fixed.csv")
metrics_summary.to_csv(METRICS_RESULTS_PATH, index=False)

print(f"✅ Metrics saved at: {METRICS_RESULTS_PATH}")


,Retriever,Strategy,Recall@5,Recall@20,Citation Precision,Citation Recall
0,E5,chain-of-thought,0.471784,0.813060,0.652059,0.912553
1,E5,few-shot,0.471784,0.813060,0.685563,0.826667
2,E5,zero-shot,0.471784,0.813060,0.659065,0.885608
3,GRITLM,chain-of-thought,0.458734,0.834072,0.611840,0.947063
4,GRITLM,few-shot,0.458734,0.834072,0.662170,0.925575
5,GRITLM,zero-shot,0.458734,0.834072,0.680844,0.919967
6,GTR,chain-of-thought,0.413363,0.765863,0.626151,0.914028
7,GTR,few-shot,0.413363,0.765863,0.663481,0.847228
8,GTR,zero-shot,0.413363,0.765863,0.669551,0.863161


✅ Metrics saved at: /data/horse/ws/uskh580e-myws/retrieval_metrics_e5_gtr_gritlm_fixed.csv


In [46]:
# Print a sample answer to debug
print("Generated Answer:", row["Generated Answer"])
print("Cited IDs:", row["Cited Corpus IDs"])

KeyError: 'Generated Answer'

In [43]:
print("Sample retrieved IDs:", retrieved_corpus_ids[:5])
print("Sample ground truth IDs:", ground_truth_corpus_ids[:5])

Sample retrieved IDs: [2239496, 253098734, 252624427, 253628210, 258832605]
Sample ground truth IDs: [258832605]


In [33]:
print("Sample data from query_df:")
display(query_df[["query", "corpusids"]].head(10))


Sample data from query_df:


,query,corpusids
0,Are there any research papers on methods to co...,[202719327]
1,Are there any resources available for translat...,[227231792]
2,Are there any studies that explore post-hoc te...,"[226254579, 204976362]"
3,Are there any tools or studies that have focus...,"[10961392, 12160022]"
4,Are there papers that propose contextualized c...,[233296182]
5,Are there studies that combine convolutional a...,[17297069]
6,Can you direct me to research that explores me...,[174801080]
7,Can you direct me to studies that explore tech...,[247593812]
8,Can you list some publications that discuss th...,"[561429, 17553490, 16509032]"
9,Can you point me to a paper that discussed tra...,[201646309]


In [31]:
# Check how many rows have empty ground truth corpus IDs
empty_ground_truth = rag_results_df["Ground Truth Corpus IDs"].apply(lambda x: len(x) == 0).sum()
total_rows = len(rag_results_df)

print(f"🚨 {empty_ground_truth} out of {total_rows} rows have empty Ground Truth Corpus IDs!")
print(f"⚠️ This means {round((empty_ground_truth / total_rows) * 100, 2)}% of rows are missing ground truth data.")


🚨 540 out of 540 rows have empty Ground Truth Corpus IDs!
⚠️ This means 100.0% of rows are missing ground truth data.
